## what does the command 'params' do?
Consider the fragment:
```julia
pars = params(Chain(...), other_flux_param);
```

How do we interpret what is going on here? Let's dig into Flux's code. I've slightly re-formatted it and added some notes below.

-----------------------
```julia
function prefor(f, x; seen = IdSet())      # IdSet is flux specific Set data structure, comparison on object-id
    x ∈ seen && return
    # Call f recursively on x (using the tree defined by the children of x).
    f(x)
    foreach(x -> prefor(f, x, seen = seen), children(x))
    return
end

function params(m)
    ps = []
    function do_things(p)
        if Tracker.istracked(p) &&
            Tracker.isleaf(p) &&  
            !any(p′ -> p′ === p, ps) 
            push!(ps, p)  
        end
    end
    prefor(do_things, m)
    return ps
end

params(m...) = params(m)
```
--------------------

So, here's what happens. `params(...)` takes an arbitrary number of arguments and slurps them into a tuple. The function then initialises an empty array as `ps` and performs the following operation the following function on the tuple:
```julia
if Tracker.istracked(p) &&    # ensure parameter is wrapped with Tracker
    Tracker.isleaf(p) &&      # ... and does not depend on anything else (no children)
    !any(p′ -> p′ === p, ps)  # ... and is not already contained in ps
    push!(ps, p)              # [THEN] add to list of parameters.
end
```
i.e., check it's tracked and doesn't actually depend on something else (which is the real parameter), and doesn't already exist. This is important as it allows us to pass in objects such as `Chain`s, which `params` can strip out the relevant parameters recursively. If it is indeed a parameter, then it gets pushed onto `ps`. As annotated above, `prefor` is simply a recursive version of `foreach`, recursing via the children of a tuple. (The tree structure is simply defined as tuples of tuples etc. .)

## Moving towards more fine grained control of parameters.

I frequently want to switch gradient learning of parameters off during optimisation, and currently have to manually set gradients to zero. The below are my first steps towards a cleaner interface.

Unfortunately since `param`s are not objects (structs), it is not easy to obtain their name, and so comparison at the moment proceeds via checking equality of the data. This is not ideal, but if it happens outside of an optimisation loop it is not a big issue.

(My train arrived before I got very far here, so this is not useful yet.)

In [1]:
struct ParGradControl
    pars::Array{Any, 1}
    mask::Array{Bool,1}
end

In [ ]:
function param_grad_control(pars::Array{Any, 1}, specified_pars::TrackedArrayOrFloat)
    mask = [specified_pars === p for p in pars]
    return ParGradControl(pars, mask)
end

function param_grad_control(pars::Array{Any, 1}, specified_pars::Array)
    mask = [[s === p for p in pars] for s in specified_pars]
    mask = reduce((x,y)-> x .| y, mask)
    return ParGradControl(pars, mask)
end

function set_relevant_zerograd(s::ParGradControl; verbose=False)
    for (m, p) in zip(s.pars, s.mask)
        m && p.grad .= 0
    end
end

function set_relevant_zerograd(s::Array; verbose=False)
    verbose && println("no ParGradControl struct")
end